In [1]:
!pip install qiskit

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 4.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 kB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [16]:
!pip install tqdm

In [17]:
import warnings; warnings.simplefilter('ignore')

from qiskit import QuantumCircuit, execute, Aer, IBMQ,QuantumRegister,ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import *

import operator


import math
import csv
import random
from qiskit import transpile, schedule as build_schedule

from tqdm import tqdm
import sys

import contextlib


In [4]:
from qiskit import IBMQ
IBMQ.save_account("2a8c9616aea0e7a684e9b4c3e2efc756847f326adbb130b86a3ccb0b6fca805fa4c2a45ba6fbea9624888efb55bc9ddfccc71add0e7599b20be354ef82f5642b")

## RQGA Configurations

In [8]:

NO_OF_QUBITS_FITNESS = 8

NO_OF_QUBITS_INDIVIDUAL = 5
POPULATION_SIZE = 2**NO_OF_QUBITS_INDIVIDUAL 
NO_OF_MAX_GROVER_ITERATIONS = int(math.sqrt(2**NO_OF_QUBITS_INDIVIDUAL))-1
#Do not change
NO_OF_QUBITS_CARRY = 2 * NO_OF_MAX_GROVER_ITERATIONS + 1

## Utility functions

In [6]:
def to_binary(value, number_of_bits, lsb=False):
    """
    Function return two's complement representation
    :param value: value in decimal representation
    :param number_of_bits: number of bits used for representation
    :returns: np.array that represents the binary representation
    >>> to_binary(10,4)
    array([1, 0, 1, 0])
    >>> to_binary(10,4,True)
    array([0, 1, 0, 1])
    """
    if lsb == True:
        return flip(array(list(binary_repr(value, number_of_bits)), dtype=int))
    return array(list(binary_repr(value, number_of_bits)), dtype=int)

In [7]:
def fitness_function():
    return 10

In [10]:
def calculate_fitness(solution):
    return 10

In [9]:
def get_random_value():
    # 2 ^ (M+1) < max < 2 ^ (M+2)
    return random.randrange(0,2**(NO_OF_QUBITS_FITNESS))

In [11]:
def get_ufit_instruction(individual_mho):
    #define and initialize the individual quantum register
    ind_qreg = QuantumRegister(NO_OF_QUBITS_INDIVIDUAL-METAHEURISTIC_GENE_LENGTH,"ind_qreg")
    #define and initialize the fitness quantum register. 
    fit_qreg = QuantumRegister(NO_OF_QUBITS_FITNESS+1,"fit_qreg")
    #create the ufit subcircuit
    qc = QuantumCircuit(ind_qreg,fit_qreg,name="U$_fit$")
    for i in range(0,POPULATION_SIZE):
        """
        For each individual in population get the two's complement representation and 
        set the qubits on 1 using X-gate, according to the binary representation
        """
        individual_binary = to_binary(i, NO_OF_QUBITS_INDIVIDUAL, True)
        
        for k in range(0,NO_OF_QUBITS_INDIVIDUAL):
            if individual_binary[k] == 0:
                qc.x(ind_qreg[k])
        """
        Calculate the fitness value and get the two's complement representation of the fitness value.
        """        
        
        valid,fitness_value = calculate_fitness(individual_binary)
        fitness_value_binary = to_binary(fitness_value,NO_OF_QUBITS_FITNESS,True)
        
        

        """
        Set the fitness value in fitness quantum register for each individual and mark it valid or invalid
        """
        for k in range(0,NO_OF_QUBITS_FITNESS):
            if fitness_value_binary[k]==1:
                qc.mct([ind_qreg[j] for j in range(0,NO_OF_QUBITS_INDIVIDUAL-METAHEURISTIC_GENE_LENGTH)],fit_qreg[k])
        #if fitness value si greater than 0 then set the valid qubit to 1
        if valid == True:
            qc.mct([ind_qreg[j] for j in range(0,NO_OF_QUBITS_INDIVIDUAL-METAHEURISTIC_GENE_LENGTH)],fit_qreg[NO_OF_QUBITS_FITNESS])
        
        
        #reset individual
        for k in range(0,NO_OF_QUBITS_INDIVIDUAL):
            if individual_binary[k] == 0:
                qc.x(ind_qreg[k])
        qc.barrier()
    return qc.to_instruction()

In [12]:
def get_oracle_instruction(positive_value_array):
    #define and initialize fitness quantum register
    fit_reg = QuantumRegister(NO_OF_QUBITS_FITNESS,"fqreg")
    #define and initialize max quantum register
    no_of_edges_reg=QuantumRegister(NO_OF_QUBITS_FITNESS,"noqreg")
    #define and initialize carry quantum register
    carry_reg = QuantumRegister(3,"cqreg")
    #define and initialize oracle workspace quantum register
    oracle = QuantumRegister(1,"oqreg")
    #create Oracle subcircuit
    oracle_circ = QuantumCircuit(fit_reg,no_of_edges_reg,carry_reg,oracle,name="O")
    
    #define majority operator
    def majority(circ,a,b,c):
        circ.cx(c,b)
        circ.cx(c,a)
        circ.ccx(a, b, c)
    #define unmajority operator
    def unmaj(circ,a,b,c):
        circ.ccx(a, b, c)
        circ.cx(c, a)
        circ.cx(a, b)
    #define the Quantum Ripple Carry Adder
    def adder_8_qubits(p,a0,a1,a2,a3,a4,a5,a6,a7,b0,b1,b2,b3,b4,b5,b6,b7,cin,cout):
        majority(p, cin, b0, a0)
        majority(p, a0, b1, a1)
        majority(p, a1, b2, a2)
        majority(p, a2, b3, a3)
        majority(p, a3, b4, a4)
        majority(p, a4, b5, a5)
        majority(p, a5, b6, a6)
        majority(p, a6, b7, a7)
        
        p.cx(a7, cout)
        
        unmaj(p, a6, b7, a7)
        unmaj(p, a5, b6, a6)
        unmaj(p, a4, b5, a5)
        unmaj(p, a3, b4, a4)
        unmaj(p, a2, b3, a3)
        unmaj(p, a1, b2, a2)
        unmaj(p, a0, b1, a1)
        unmaj(p, cin, b0, a0)
    
    """
    Subtract max value. We start by storing the max value in the quantum register. Such, considering that 
    all qubits are |0>, if on position i in positive_value_array there's 0, then qubit i will be negated. Otherwise, 
    if on position i in positive_value_array there's a 1, by default will remain 0 in no_of_edges_reg quantum
    register. For performing subtraction, carry-in will be set to 1.
    """
    for i in range(0,NO_OF_QUBITS_FITNESS):
        if positive_value_array[i]==0:
            oracle_circ.x(no_of_edges_reg[i])
    oracle_circ.x(carry_reg[0])

    adder_8_qubits(oracle_circ, 
            no_of_edges_reg[0],no_of_edges_reg[1],no_of_edges_reg[2],no_of_edges_reg[3],
            no_of_edges_reg[4],no_of_edges_reg[5],no_of_edges_reg[6],no_of_edges_reg[7],       
            fit_reg[0],fit_reg[1],fit_reg[2],fit_reg[3],
            fit_reg[4],fit_reg[5],fit_reg[6],fit_reg[7],
               carry_reg[0],carry_reg[1]);

    
    oracle_circ.barrier()
    """
    Reset the value in no_of_edges_reg and carry-in
    """
    oracle_circ.x(no_of_edges_reg)
    oracle_circ.x(carry_reg[0])
    
    """
    Mark the corresponding basis states by shifting their amplitudes.
    """
    
    oracle_circ.h(oracle[0])
    oracle_circ.mct([fit_reg[i] for i in range(0,NO_OF_QUBITS_FITNESS)],oracle[0])
    oracle_circ.h(oracle[0])
    
    """
    Restore the fitness value by adding max value.
    """
    adder_8_qubits(oracle_circ, 
            no_of_edges_reg[0],no_of_edges_reg[1],no_of_edges_reg[2],no_of_edges_reg[3],
            no_of_edges_reg[4],no_of_edges_reg[5],no_of_edges_reg[6],no_of_edges_reg[7],       
            fit_reg[0],fit_reg[1],fit_reg[2],fit_reg[3],
            fit_reg[4],fit_reg[5],fit_reg[6],fit_reg[7],
            carry_reg[0],carry_reg[2]);
    return oracle_circ.to_instruction()

In [13]:
def get_grover_iteration_subcircuit():
    #define and initialize fitness quantum register
    fit_qreg = QuantumRegister(NO_OF_QUBITS_FITNESS+1,"fqreg")
    #define and initialize oracle workspace quantum register
    oracle_ws = QuantumRegister(1,"ows")
    #create grover diffuser subcircuit
    grover_circ = QuantumCircuit(fit_qreg,oracle_ws,name ="U$_s$")

    grover_circ.h(fit_qreg)
    grover_circ.x(fit_qreg)

    grover_circ.h(oracle_ws[0])

    grover_circ.mct(list(range(NO_OF_QUBITS_FITNESS+1)), oracle_ws[0])  # multi-controlled-toffoli

    grover_circ.h(oracle_ws[0])


    grover_circ.x(fit_qreg)
    grover_circ.h(fit_qreg)
    grover_circ.h(oracle_ws)

    return grover_circ.to_instruction()


In [14]:
def run_algorithm(run_no, writer):
    #Load IBMQ account
    IBMQ.load_account()
    #calculate the number of edges in graph
    max_value = 0
    new_fitness_value = get_random_value()
    print("Max value:{0}".format(new_fitness_value))
    #define a list for storing the results
    final_results = []
    
    number_of_ga_generations = 0

    print("Preparing quantum registers and creating quantum circuit...")
    ind_qreg = QuantumRegister(NO_OF_QUBITS_INDIVIDUAL,"ireg")
    fit_qreg = QuantumRegister(NO_OF_QUBITS_FITNESS+1,"freg")
    carry_qreg = QuantumRegister(NO_OF_QUBITS_CARRY,"qcarry")
    oracle = QuantumRegister(1,"oracle")
    creg = ClassicalRegister(NO_OF_QUBITS_INDIVIDUAL,"reg")
    max_value_qreg = QuantumRegister(NO_OF_QUBITS_FITNESS,"max_value_qreg")

    print("Creating quantum circuit...")

    qc = QuantumCircuit(ind_qreg,fit_qreg,carry_qreg,oracle,max_value_qreg,creg)

    print("Creating superposition of individuals...")
    qc.h(ind_qreg)
    qc.h(oracle)
    
    print("Getting ufit, oracle and grover iterations subcircuits...")
    ufit_instr = get_ufit_instruction(individual_mho)
  

    print("Append Ufit instruction to circuit...")
    qc.append(ufit_instr, [ind_qreg[q] for q in range(0,NO_OF_QUBITS_INDIVIDUAL)]+
                          [fit_qreg[q] for q in range(0,NO_OF_QUBITS_FITNESS+1)]
            )
    generation = 0
    while new_fitness_value != max_value and generation < 8:
        max_value = new_fitness_value
        max_value_bin = to_binary(max_value,NO_OF_QUBITS_FITNESS,True)
        oracle_instr = get_oracle_instruction(max_value_bin)
        grover_iter_inst = get_grover_iteration_subcircuit()
        for it in range(0,NO_OF_MAX_GROVER_ITERATIONS):
            print("Append Oracle instruction to circuit...")

            qc.append(oracle_instr,[fit_qreg[q] for q in range(0,NO_OF_QUBITS_FITNESS)]+
                               [max_value_qreg[q] for q in range(0,NO_OF_QUBITS_FITNESS)]+
                               [carry_qreg[0],carry_qreg[2*it+1],carry_qreg[2*it+2]]+
                               [oracle[0]]
                     )
            print("Append Grover Diffuser to circuit...")
            qc.append(grover_iter_inst, [fit_qreg[q] for q in range(0,NO_OF_QUBITS_FITNESS+1)]+
                                        [oracle[0]]
                     )

        print("Measure circuit...")
        qc.measure(ind_qreg,creg)

        simulation_results = []

        provider = IBMQ.get_provider(hub='ibm-q',group='open', project='main')
        backend = provider.get_backend('simulator_mps')
        # Perform 10 measurements for each circuit
       
        print("Setup simulator...")    
        shots = 16
        try:
            print("Starting simulator...")
            mapped_circuit = transpile(qc, backend=backend)
            qobj = assemble(mapped_circuit, backend=backend, shots=shots)
            runner = backend.run(qobj)
            job_monitor(runner)
            results = runner.result()
            answer = results.get_counts()
            #Get the result with the maximum number of counts
            max_item =max(answer.items(), key=operator.itemgetter(1))
            solution_individual=string_to_list(max_item[0])
            solution_individual=recreate_individual(individual_mho,solution_individual)
            _,new_fitness_value=calculate_individual_fitness(solution_individual)
            new_fitness_value = abs(new_fitness_value)
            writer.writerow({"algo_run_no":run_no,"solution":solution_individual, "fitness_value":new_fitness_value,"rqga_generation":generation,"ga_generation":number_of_ga_generations})

            print("Found solution {0} with fitness {1}...".format(solution_individual,new_fitness_value))
        except Exception as e:
            print(str(e))
        generation+=1
        qc.reset(max_value_qreg)

## RUN

In [ ]:


with open("results.csv", "w", newline='') as csvfile:
    fieldnames = ["algo_run_no", "solution", "fitness_value","rqga_generation", "ga_generation"]
    writer = csv.DictWriter(csvfile,fieldnames=fieldnames)
    writer.writeheader()

    for i in tqdm(range(100)):
        run_algorithm(i,writer)